## Load the necessary modules

Zipline only works with an older version of pandas. It is necessary to downgrade to the version of pandas that is 0.22.0

In [1]:
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import zipline
from zipline.api import order, record, symbol
from zipline.assets import Equity, Future
from zipline import run_algorithm
from collections import OrderedDict
from datetime import datetime

## Load and Prelimenary Analysis of Data

In [ ]:
# Store the different datasets
data = OrderedDict()
# Loads the data from the csv file
stocks  = ["AAPL", "AMZN", "BA", "FB", "GOOG", "MA", "MSFT", "NVDA", "UNH", "V"]

# Iterate to load stock data from csv
for stock in stocks:
    data[stock] = pd.read_csv("{}.csv".format(stock), index_col =0, parse_dates=['Date'])
    data[stock] = data[stock][["open","high","low","close","volume"]]
    print(data[stock].head(10))

*Price of the stock of the ten companies of interest from 2013 to 2018*

In [ ]:
plt.figure(figsize=(15,10))
for stock in stocks:
    data[stock]['close'].plot()
plt.legend(stocks)
plt.ylabel("Price in $")

# Building our portfolio

In [ ]:
# Calculate the weekly and yearly returns for stocks
data

## Make data compatible with Zipline

In [ ]:
# Convert data to panel
StockPrice = pd.Panel(data)
StockPrice.minor_axis = ["open","high","low","close","volume"]
StockPrice.major_axis = StockPrice.major_axis.tz_localize(pytz.utc)
print(StockPrice)

In [ ]:
# I dont think this does anything
%load_ext zipline

## Set constraints 

In [ ]:
# Constraints for our algorithm
initial_capital = 10000
# Time frame for our algorithm
startDate = datetime(2013,1,2,0,0,0,0,pytz.utc)
endDate = datetime(2018,12,28,0,0,0,0,pytz.utc)
# Cost per trade will be 4.95 just because of my brokerage
DEFAULT_MINIMUM_COST_PER_EQUITY_TRADE = 4.95
DEFAULT_MINIMUM_COST_PER_FUTURE_TRADE = 4.95


## Run Algorithm

In [ ]:
# Define necessary two functions to run algorithm
def initialize(context):
    context.assest = symbols('GOOG')
    
def handle_data(context, data):
    order(symbol('GOOG'),10)
    record(GOOG = data.current(symbol('GOOG'),'price'))



In [ ]:
perf = zipline.run_algorithm(start= startDate,
                            end= endDate,
                            initialize=initialize,
                            data_frequency='daily',
                            capital_base= initial_capital,
                            handle_data=handle_data,
                            data=StockPrice)
perf.head(5)